<a href="https://colab.research.google.com/github/manashpratim/Language-Modelling-using-RNNs/blob/master/HW4P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline

import numpy as np
from matplotlib import pyplot as plt
import time
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from tests import test_prediction, test_generation

In [0]:
# load all that we need

dataset = np.load('../dataset/wiki.train.npy',allow_pickle=True)
valid = np.load('../dataset/wiki.valid.npy',allow_pickle=True)
fixtures_pred = np.load('../fixtures/prediction.npz',allow_pickle=True)  # dev
fixtures_gen = np.load('../fixtures/generation.npy',allow_pickle=True)  # dev
fixtures_pred_test = np.load('../fixtures/prediction_test.npz',allow_pickle=True)  # test
fixtures_gen_test = np.load('../fixtures/generation_test.npy',allow_pickle=True)  # test
vocab = np.load('../dataset/vocab.npy',allow_pickle=True)

In [0]:
# data loader

class LanguageModelDataLoader(DataLoader):
    """
        TODO: Define data loader logic here
    """
    def __init__(self, dataset, batch_size, shuffle=True):
        
        self.data = dataset
        self.batch_size = batch_size
        self.seq_len = 70
        self.shuffle = shuffle

    def __iter__(self):
        # concatenate your articles and build into batches
        
        if self.shuffle == True:
            np.random.shuffle(self.data)
        text = self.data[0]
        for i in range(1,len(self.data)):
            text = np.hstack((text,self.data[i]))
        
        n_seq = len(text) // self.seq_len
        text = text[:n_seq * self.seq_len]
        self.batch_data = torch.tensor(text,dtype=torch.long).view(-1,self.seq_len)
        
        
        for i in range(0,self.batch_data.shape[0],self.batch_size):
    
            yield (self.batch_data[i:i+self.batch_size,:-1].T, self.batch_data[i:i+self.batch_size,1:].T)
     

In [0]:
# model

class LanguageModel(nn.Module):
    """
        TODO: Define your model here
    """
    def __init__(self, vocab_size):
        super(LanguageModel, self).__init__()
        

        self.vocab = vocab_size
        self.embed = nn.Embedding(vocab_size, 1000)
        self.lstm = nn.LSTM(1000, 1024)
        self.output = nn.Linear(1024, vocab_size)


    def forward(self, x):

        batch_size = x.shape[1]
        embed = self.embed(x)
        output_lstm,_ = self.lstm(embed)
        output_lstm_flatten = output_lstm.view(-1,1024) 
        output_flatten = self.output(output_lstm_flatten) 
        
        return output_flatten.view(-1,batch_size,self.vocab)
    
    def generate(self,seq, n_words): # L x V

        generated_words = []
        embed = self.embed(seq).unsqueeze(1)
        output_lstm, _ = self.lstm(embed) 
        output1 = output_lstm[-1]
        scores = self.output(output1) 
        _,current_word = torch.max(scores,dim=1) 
        generated_words.append(current_word)
        if n_words > 1:
            for i in range(n_words-1):
                embed = self.embed(current_word).unsqueeze(0) 
                output_lstm,_ = self.lstm(embed) 
                output2 = output_lstm[0] 
                scores = self.output(output2) 
                _,current_word = torch.max(scores,dim=1) 
                generated_words.append(current_word)
        return torch.cat(generated_words,dim=0)

In [0]:
# model trainer

class LanguageModelTrainer:
    def __init__(self, model, loader, max_epochs=1, run_id='exp'):
        """
            Use this class to train your model
        """

        self.model = model
        self.loader = loader
        self.train_losses = []
        self.val_losses = []
        self.predictions = []
        self.predictions_test = []
        self.generated_logits = []
        self.generated = []
        self.generated_logits_test = []
        self.generated_test = []
        self.epochs = 0
        self.max_epochs = max_epochs
        self.run_id = run_id
        
        self.optimizer = torch.optim.Adam(model.parameters(),lr=0.001, weight_decay=1e-6)
        self.criterion = nn.CrossEntropyLoss()

    def train(self):
        self.model.train() # set to training mode
        epoch_loss = 0
        num_batches = 0
        for batch_num, (inputs, targets) in enumerate(self.loader):
            epoch_loss += self.train_batch(inputs, targets)
            #if batch_num%10==0:
            #    print(batch_num)
        epoch_loss = epoch_loss / (batch_num + 1)
        self.epochs += 1
        print('[TRAIN]  Epoch [%d/%d]   Loss: %.4f'
                      % (self.epochs + 1, self.max_epochs, epoch_loss))
        self.train_losses.append(epoch_loss)

    def train_batch(self, inputs, targets):
        """ 
            TODO: Define code for training a single batch of inputs
        
        """
        outputs = model(inputs) # 3D
        loss = self.criterion(outputs.reshape(-1,outputs.size(2)),targets.reshape(-1)) # Loss of the flattened outputs
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.item()

    
    def test(self):

        self.model.eval() # set to eval mode
        predictions = TestLanguageModel.prediction(fixtures_pred['inp'], self.model) # get predictions
        self.predictions.append(predictions)
        generated_logits = TestLanguageModel.generation(fixtures_gen, 10, self.model) # generated predictions for 10 words
        generated_logits_test = TestLanguageModel.generation(fixtures_gen_test, 10, self.model)
        nll = test_prediction(predictions, fixtures_pred['out'])
        generated = test_generation(fixtures_gen, generated_logits, vocab)
        generated_test = test_generation(fixtures_gen_test, generated_logits_test, vocab)
        self.val_losses.append(nll)
        
        self.generated.append(generated)
        self.generated_test.append(generated_test)
        self.generated_logits.append(generated_logits)
        self.generated_logits_test.append(generated_logits_test)
        
        # generate predictions for test data
        predictions_test = TestLanguageModel.prediction(fixtures_pred_test['inp'], self.model) # get predictions
        self.predictions_test.append(predictions_test)
            
        print('[VAL]  Epoch [%d/%d]   Loss: %.4f'
                      % (self.epochs + 1, self.max_epochs, nll))
        return nll

    def save(self):

        model_path = os.path.join('experiments', self.run_id, 'model-{}.pkl'.format(self.epochs))
        torch.save({'state_dict': self.model.state_dict()},
            model_path)
        np.save(os.path.join('experiments', self.run_id, 'predictions-{}.npy'.format(self.epochs)), self.predictions[-1])
        np.save(os.path.join('experiments', self.run_id, 'predictions-test-{}.npy'.format(self.epochs)), self.predictions_test[-1])
        np.save(os.path.join('experiments', self.run_id, 'generated_logits-{}.npy'.format(self.epochs)), self.generated_logits[-1])
        np.save(os.path.join('experiments', self.run_id, 'generated_logits-test-{}.npy'.format(self.epochs)), self.generated_logits_test[-1])
        with open(os.path.join('experiments', self.run_id, 'generated-{}.txt'.format(self.epochs)), 'w') as fw:
            fw.write(self.generated[-1])
        with open(os.path.join('experiments', self.run_id, 'generated-{}-test.txt'.format(self.epochs)), 'w') as fw:
            fw.write(self.generated_test[-1])


In [0]:
class TestLanguageModel:
    def prediction(inp, model):
        """
            TODO: write prediction code here
            
            :param inp:
            :return: a np.ndarray of logits
        """
        outputs = model(torch.tensor(inp,dtype=torch.long).T)

        return outputs[-1].detach().numpy()

        
    def generation(inp, forward, model):
        """
            TODO: write generation code here

            Generate a sequence of words given a starting sequence.
            :param inp: Initial sequence of words (batch size, length)
            :param forward: number of additional words to generate
            :return: generated words (batch size, forward)
        """ 
        a = np.zeros((inp.shape[0],forward),dtype=int)
        inpu = torch.tensor(inp,dtype=torch.long).T
        for i in range(inp.shape[0]):
            
            a[i] = model.generate(inpu[:,i], forward).detach().numpy()
            
        #raise NotImplemented
        return a
        

In [0]:
# TODO: define other hyperparameters here

NUM_EPOCHS = 2
BATCH_SIZE = 64


In [0]:
run_id = str(int(time.time()))
if not os.path.exists('./experiments'):
    os.mkdir('./experiments')
os.mkdir('./experiments/%s' % run_id)
print("Saving models, predictions, and generated words to ./experiments/%s" % run_id)

Saving models, predictions, and generated words to ./experiments/1586644087


In [0]:
model = LanguageModel(len(vocab))
loader = LanguageModelDataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=True)
trainer = LanguageModelTrainer(model=model, loader=loader, max_epochs=NUM_EPOCHS, run_id=run_id)

In [0]:
best_nll = 1e30 
for epoch in range(NUM_EPOCHS):
    trainer.train()
    nll = trainer.test()
    if nll < best_nll:
        best_nll = nll
        print("Saving model, predictions and generated output for epoch "+str(epoch)+" with NLL: "+ str(best_nll))
        trainer.save()
    

In [0]:

plt.figure()
plt.plot(range(1, trainer.epochs + 1), trainer.train_losses, label='Training losses')
plt.plot(range(1, trainer.epochs + 1), trainer.val_losses, label='Validation losses')
plt.xlabel('Epochs')
plt.ylabel('NLL')
plt.legend()
plt.show()